In [2]:
from mc_estimation import *
from model_dynamics import *
from data.experience_history import ExperienceHistory
import tensorflow as tf

In [5]:
data_buff = ExperienceHistory(num_frame_stack=4, capacity=int(1e6), pic_size=(96,96))
expert_data_path = "../data/expert/demos.pkl"
data_buff.loadExpertHistory(expert_data_path)

Expert Data loaded with  1599  frames. 1599


In [3]:
'''Markov Chain'''
mc_config = {  
            "loss": tf.keras.losses.MSE, 
            "optimizer": tf.keras.optimizers.Adam(learning_rate=0.00001),
            "epochs": 1,
            "batch_size": 64
             }
mc_model = MarkovChainCNNModel(input_dim = (96,96, 4))
trainMarkovChainCNN(model=mc_model, experience_history=data_buff, config=mc_config)

'''Transition Probs'''
trans_config = {  
            "loss": tf.keras.losses.MSE, 
            "optimizer": tf.keras.optimizers.Adam(learning_rate=0.00001),
            "epochs": 1,
            "batch_size": 64
             }
trans = TransitionModel(input_dim=(96,96,4), action_dim=12)
# trainTransitions(model=trans, experience_history=data_buff, config=trans_config)



In [6]:
'''policy and train'''
policy = CnnPolicy(action_dim=12)
mc_est_config = {  
        "loss": tf.keras.losses.KLD, 
        "optimizer": tf.keras.optimizers.Adam(learning_rate=0.00001),
        "validation_split": 0.10,
        "max_obs": int(1e5),
        "epochs": 16,
        "batch_size": 64,
        "state_size": (96,96,4), 
        "action_size": 12
        }
mc_est = imageMcEstimator(config=mc_est_config, mc_model=mc_model, dynamics_model=trans, policy=policy, dataset=data_buff)
mc_est.train_mc()

Batch: 3/1599 - Loss: 0.0032880362

KeyboardInterrupt: 

In [5]:
policy.save("./models/cr_policy")
# mc_model.save("./models/cr_mc")
# trans.save("./models/cr_trans")

INFO:tensorflow:Assets written to: ./models/cr_policy/assets


In [6]:
policy_l = tf.keras.models.load_model("./models/cr_policy")

In [14]:
policy_l(np.random.rand(1,96,96,4))
a = np.random.rand(96,96,4)
b = np.random.rand(96,96,1)
c = np.concatenate((a,b), axis=2)
c.shape

(96, 96, 5)

In [21]:
'''Action Map'''
from skimage import color, transform
import itertools as it
action_map = np.array([k for k in it.product([-1, 0, 1], [1, 0], [0.2, 0])])

'''Initial Buffer'''
buff = np.random.rand(96,96,4)

def process_image(obs):
    return 2 * color.rgb2gray(obs) - 1.0

import gym
env_name = "CarRacing-v0"
env = gym.make(env_name)

'''Test Performance of Agent'''
observation = env.reset()
reward_total = 0
reward_history = []
steps = 0
for _ in range(1000):
    env.render()
    # action = env.action_space.sample() # your agent here (this takes random actions)
    print(observation.shape)
    
    observation = np.reshape(np.array([process_image(observation)]), newshape=(96,96,1))
    buff = np.concatenate((observation, buff[:,:,0:3]), axis=2)
    # print(observation)
    # observation = dynamics_data.transform_obs_forward(observation)
    # print(observation)
    # print()
    action_choices = policy_l(np.array([buff]))
    # print(action_choices)
    action = np.argmax(action_choices)
    action = action_map[action]
    steps += 1
    # print("Action taken: " + str(action))
    observation, reward, done, info = env.step(action)
    reward_total += reward

    if done:
        observation = env.reset()
        print("Reward Total: " + str(reward_total))
        print("Reset")
        print("steps: " + str(steps))
        reward_history.append(reward_total)
        reward_total=0
        steps = 0

/home/sasha/.local/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
Track generation: 1240..1562 -> 322-tiles track
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96

In [23]:
env.close()